<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/15regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
### Rafa Caballero

## Regresión lineal

### Índice
[Obtención del modelo](#modelo)<br>
[Afinando el error](#error)<br>
[Intervalo de confianza](#intervalo)<br>
[Mejorando el modelo](#mejorando)<br>
[Lasso y Ridge](#lasso)<br>



Antes de empezar comprobamos la versión de sklearn

In [ ]:
import sklearn
sklearn.__version__

Si se quiere actualizar ejecutar el siguiente código (y después Kernel - Restart)

In [ ]:
#!pip  install scikit-learn --upgrade --user

<a name="modelo"></a>
## Obtención del modelo

Empezamos cargando un fichero con las notas de las pruebas PISA. Recordamos la importancia del preprocesado, pero es aquí nos dan ya los datos preparados para que nos centremos en la regresión

In [ ]:
# Paso 0: Carga y preparación del fichero
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)
df

1.- Dividir las columnas en X e y e. Nuestro objetivo será deducir la columna MAT (y) desde dos columnas: SCI y REA, que serán las X

In [ ]:

XColumns = ["SCI", "REA"] # una lista de columnas: las X, las características, las features, las dimensiones...
yColumn = "MAT"  # la y, siempre una única columna
# dividimos en dos; X será un dataframe, y una columna (tipo Series)
X = df[XColumns]
y = df[yColumn]


2.- Dividir las columnas en X e y en train y test. En este caso ponemos 70% para train y 30% para test

In [ ]:

from sklearn.model_selection import train_test_split

test = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

Tendremos que

*   X = X_train + X_test
*   y = y_train + y_test


In [ ]:
X

In [ ]:
X_train

In [ ]:
X_test

3.- Declarar el método y  entrenar con el conjunto train, obteniendo un *modelo*

In [ ]:
from sklearn.linear_model import LinearRegression
metodo = LinearRegression()
modelo = metodo.fit(X_train,y_train)

El modelo representa simplemente la recta que mejor ajusta las (X,y) dadas:

In [ ]:
modelo.intercept_, modelo.coef_

In [ ]:
print(f"La recta de regresión es {yColumn} =\
 {round(modelo.coef_[0],3)}*{XColumns[0]} + {round(modelo.coef_[1],3)}*{XColumns[1]} + \
 {modelo.intercept_}")

Aunque veremos que estas predicciones las vamos a hacer automáticamente, podríamos escribir por nuestra cuentra una función que las haga:

In [ ]:
def y_predict(X,modelo):
    s = modelo.intercept_
    for i,x in enumerate(X):
        s += x*modelo.coef_[i]
    return s

y_predict([400,450],modelo)

Vamos a representar gráficamente los datos de entrenamiento y la recta modelo

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib

fig = plt.figure(figsize=(14,6))

# projection='3d' indica que este subplot es en 3d
ax = fig.add_subplot(1, 2, 1, projection='3d')
x = X_train["SCI"]
y = X_train["REA"]
z = y_train
ax.scatter3D( x, y, z)

y_1 = y_predict([300,300],modelo)
y_2 = y_predict([600,600],modelo)
ax.plot3D([300,600],[300,600],[y_1,y_2],color="green")
ax.set_zlabel(r"MAT", fontsize=10, color="blue")
ax.set_xlabel(r"SCI", fontsize=10, color="blue")
ax.set_ylabel(r"REA", fontsize=10, color="blue")
plt.show()

4.- Ahora predecimos con el test y mostramos el error

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

Y completamos la figura mostrando los puntos de entrenamiento, de test y la recta

In [ ]:
fig = plt.figure(figsize=(14,6))

# projection='3d' indica que este subplot es en 3d
ax = fig.add_subplot(1, 2, 1, projection='3d')
x = X_train["SCI"]
y = X_train["REA"]
z = y_train
ax.scatter3D( x, y, z,label="train")

y_1 = y_predict([300,300],modelo)
y_2 = y_predict([600,600],modelo)
ax.plot3D([300,600],[300,600],[y_1,y_2],color="green")
ax.set_zlabel(r"MAT", fontsize=10, color="blue")
ax.set_xlabel(r"SCI", fontsize=10, color="blue")
ax.set_ylabel(r"REA", fontsize=10, color="blue")


##### Esto es lo nuevo
x = X_test["SCI"]
y = X_test["REA"]
z = y_test
ax.scatter3D( x, y, z,label="test")
plt.legend()

plt.show()

**Ejercicio 1** Repetir los pasos 1,2,3,4, cambiando tan solo el tamaño del test a 0.95 ¿qué sucede con el error?


Este es un ejemplo de overfitting: cuando por alguna razón (en este caso por falta de datos de entrenamiento), el modelo generado se comporta mucho mejor sobre el entrenamiento que sobre el test. Repitamos todo el proceso:

<a name="error"></a>


## Afinando el error

**Ejercicio 2** Ejecutar varias veces el código anterior, se verá que se obtiene resultados diferentes ¿en qué punto del código se produce esta variación?

De hecho, si se quieren obtener resultados con una cierta verosimilitud tendremos que repetir el experimento (pasos 2,3,4) varias veces. Vamos a instalar el paquete progress bar para ver cómo progresan los experimentos

In [ ]:
!pip install tqdm

In [ ]:
# Carga del fichero
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math

from tqdm import tqdm

url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1
XColumns = ["SCI", "REA"]
yColumn = "MAT"
X = df[XColumns]
y = df[yColumn]

veces = 500


resultados = []
for v in tqdm(range(veces)):
    # 2
    test = 0.4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

    # 3
    metodo = LinearRegression()
    modelo = metodo.fit(X_train,y_train)

    # 4
    y_pred = modelo.predict(X_test)
    r2 = r2_score(y_test,y_pred)
    rmse = math.sqrt(mean_squared_error(y_test,y_pred))
    mae = mean_absolute_error(y_test,y_pred)
    bias = (y_test - y_pred).mean()
    resultados.append([round(r2,3),round(rmse,3),round(mae,3),round(bias,3)])
    #print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

df_errores = pd.DataFrame(resultados,columns=["r^2","RMSE","MAE","BIAS"])
df_errores

In [ ]:
df_errores.describe()

In [ ]:
df_errores.describe().loc["mean"]

Es fácil hacer una función que haga el trabajo de los experimentos. Podemos usarla para evaluar la regresión con otros datos

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
from tqdm import tqdm

def evalua_regresion(df,XColumns,yColumn,veces=500):
    # 1
    X = df[XColumns]
    y = df[yColumn]



    resultados = []
    for v in tqdm(range(veces)):
        # 2
        test = 0.4
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

        # 3
        metodo = LinearRegression()
        modelo = metodo.fit(X_train,y_train)

        # 4
        y_pred = modelo.predict(X_test)
        r2 = r2_score(y_test,y_pred)
        rmse = math.sqrt(mean_squared_error(y_test,y_pred))
        mae = mean_absolute_error(y_test,y_pred)
        bias = (y_test - y_pred).mean()
        resultados.append([round(r2,3),round(rmse,3),round(mae,3),round(bias,3)])
        #print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

    df_errores = pd.DataFrame(resultados,columns=["r^2","RMSE","MAE","BIAS"])
    return df_errores.describe().loc["mean"]


In [ ]:
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1
XColumns = ["SCI", "REA"]
yColumn = "MAT"
print(evalua_regresion(df,XColumns,yColumn))

**Ejercicio 3**

- Si se en lugar de predecir "MAT" desde "SCI" y "REA" tuviéramos que predecir "SCI" desde las otras dos o "REA" desde las otras dos cual daría mejores resultados
- Si solo queremos utilizar un atributo, ya sea "SCI" o "REA" para predecir "MAT" ¿cuál usarías?

<a name="intervalo"></a>
## Intervalos de confianza

Si ahora ejecutamos varias veces el código veremos que los resultados son bastante estables. Aun así persiste una duda...cuando hagamos una predicción ¿podemos estimar lo lejos que está del valor real?

Vamos a poder hacerlo con un par de condiciones:

1.- Que el bias sea muy próximo a 0

2.- Que el RMSE sea una normal

La condición 1 la tenemos; hay que tener en cuenta que un valor de alrededor de -0.1...0.1 en

In [ ]:
df.MAT.mean()

es muy pequeño. En cuanto al RMSE por simplificar nos conformamos con "ver" el histograma

In [ ]:
import matplotlib.pyplot as plt
#fig, ax = plt.subplots(figsize=(5, 5))
df_errores.RMSE.hist(bins=15)
plt.show()

Asumiendo que esto es una normal, podemos decir que, con un 95% de probabilidades, para cualquier predicción p el valor real estará en el intervalo

[p -2RMSE, p+2RMSE]

Este intervalo de confianza se mantiene siempre y cuando estemos (aprox.) dentro del rango de valores usados en el entrenamiento

In [ ]:
X_train.describe()

**Ejemplo**

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1
XColumns = ["SCI", "REA"]
yColumn = "MAT"
X = df[XColumns]
y = df[yColumn]

metodo = LinearRegression()
modelo = metodo.fit(X.values,y)

p = modelo.predict([[400,450]])[0]

p

Como el RMSE es aproximadamente 11.75, tendremos que con un 95% de probabilidades el valor real está entre

In [ ]:
RMSE = 11.75
(p-1.96*RMSE,p+1.96*RMSE)

In [ ]:
import matplotlib.pyplot as plt
df[["MAT"]].hist()
plt.plot([p-1.96*RMSE,p-1.96*RMSE],[0,1],color="pink",linewidth=5)
plt.plot([p+1.96*RMSE,p+1.96*RMSE],[0,1],color="pink",linewidth=5)
plt.plot([p,p],[0,1],color="yellow",linewidth=5)
plt.show()

Sin embargo, los siguientes diagramas de residuos nos indican que algo no está funcionando como esperábamos

In [ ]:
X = df[XColumns]
y = df[yColumn]

x = range(len(y))
y_pred = modelo.predict(X.values)
plt.scatter(x,y_pred,color="red",s=1)
plt.scatter(x,y,color="blue",s=8)
for i,v in enumerate(y_pred):
    plt.plot([x[i],x[i]], [v-2*RMSE,v+2*RMSE],color="green")
#for y_v in y_pred:

In [ ]:
x = range(len(y))
fig, ax = plt.subplots(figsize=(15, 5))
y_pred = modelo.predict(X.values)
ci = 1.96*RMSE
for i,v in enumerate(y_pred):
    plt.plot([x[i],x[i]], [v,y[i]],color="green")
ax.fill_between(x, ( y_pred-ci), ( y_pred+ci), color='b', alpha=.1)
ax.scatter(x,y_pred,color="red",s=8)
ax.scatter(x,y,color="blue",s=8)
plt.show()

In [ ]:
y_pred = metodo.predict(X.values)

x_plot = plt.scatter(y_pred, (y_pred - y), c='b')
plt.hlines(y=0, xmin= -1, xmax=800)

plt.hlines(y=2*RMSE, xmin= -1, xmax=800,color="r")
plt.hlines(y=-2*RMSE, xmin= -1, xmax=800,color="r")
plt.show()

<a name="mejorando"></a>
## Mejorando el modelo

El histograma nos sugiere que quizás el modelo sea mejor si dividimos el conjunto en dos

In [ ]:
df2 = df[df[yColumn]<450]
X = df2[XColumns]
y = df2[yColumn]

metodo = LinearRegression()
modelo = metodo.fit(X.values,y)

r2,RMSE,mae,bias = evalua_regresion(df2,XColumns,yColumn)
print(RMSE)

x = range(len(y))
y_pred = modelo.predict(X.values)
plt.scatter(x,y_pred,color="red",s=1)
plt.scatter(x,y,color="blue",s=8)
for i,v in enumerate(y_pred):
    plt.plot([x[i],x[i]], [v-2*RMSE,v+2*RMSE],color="green")

In [ ]:
x_plot = plt.scatter(y_pred, (y_pred - y), c='b')
plt.hlines(y=0, xmin= -1, xmax=800)

plt.hlines(y=2*RMSE, xmin= -1, xmax=800,color="r")
plt.hlines(y=-2*RMSE, xmin= -1, xmax=800,color="r")
plt.show()

In [ ]:
df2 = df[df[yColumn]>=450]
X = df2[XColumns]
y = df2[yColumn]

metodo = LinearRegression()
modelo = metodo.fit(X.values,y)

r2,RMSE,mae,bias = evalua_regresion(df2,XColumns,yColumn)
print(RMSE)

x = range(len(y))
y_pred = modelo.predict(X.values)
plt.scatter(x,y_pred,color="red",s=1)
plt.scatter(x,y,color="blue",s=8)
for i,v in enumerate(y_pred):
    plt.plot([x[i],x[i]], [v-2*RMSE,v+2*RMSE],color="green")

In [ ]:
x_plot = plt.scatter(y_pred, (y_pred - y), c='b')
plt.hlines(y=0, xmin= -1, xmax=800)

plt.hlines(y=2*RMSE, xmin= -1, xmax=800,color="r")
plt.hlines(y=-2*RMSE, xmin= -1, xmax=800,color="r")
plt.show()

<a name="lasso"></a>
##  Ridge y Lasso

Variantes de la regresión, resultan son técnicas de regularización utilizadas para prevenir el sobreajuste en modelos de regresión lineal. Tienen características ligeramente diferentes:

Ridge: Permite reducir la varianza en presencia de multicolinealidad manteniendo todos los predictores.

Lasso: Selecciona de variables (a algunas les puede dar coeficientes 0 --> regularización más extrema y parsimonioso) y obtener un modelo más simple y interpretable.

In [ ]:
# Carga del fichero
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1
XColumns = ["SCI", "REA"]
yColumn = "MAT"
X = df[XColumns]
y = df[yColumn]

# 2
from sklearn.model_selection import train_test_split
test = 0.95
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3
from sklearn.linear_model import LinearRegression
metodo = LinearRegression()
modelo = metodo.fit(X_train,y_train)

# 4
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

In [ ]:
from sklearn.linear_model import Lasso
modelo = Lasso(alpha=6).fit(X_train, y_train)
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

In [ ]:
from sklearn.linear_model import Ridge
modelo = Ridge(alpha=6).fit(X_train, y_train)
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

ElasticNet combina los dos

In [ ]:
from sklearn.linear_model import ElasticNet
modelo = ElasticNet(alpha=6, l1_ratio=0.5).fit(X_train, y_train)
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")